# GT2021-60283 Demo IV - Tune BNN with Random Search

In [ ]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers, Model, Sequential
from tensorflow import keras
from tensorflow.keras import backend as K
#from dynamicsML.NN import Tuner, setup_NN
from sklearn.preprocessing import StandardScaler
from subprocess import Popen
import seaborn as sns
import random
import time
import psutil as ps
import glob
%matplotlib inline


The notebook runs the `BNN_train.py` script with different combinations of hidden layers, hidden layer neurons and learning rates in order to tune the BNN on the given data. Random selections of hidden layers an hidden layer nueons will be made and then the learning rate will be decreased until the minimum loss occurs after 200 epochs. After each trial the history will be saved and loaded into a summary data frame which will track the trials ad can be used to find the optimum combination.

In [ ]:
def get_running():
    running = []
    for proc in ps.process_iter():
        try:
            if 'BNN_train.py' in proc.cmdline():
                run_no = proc.cmdline()[-4]
                if run_no not in running:
                    running.append(run_no)
        except:
            pass
    return running
def run_trial(num_hidden, num_neurons, indep_div, lr, run_id, single_run_mode=False, NNs=8, gpu=True, dataset_name="NTNU0000_FFT", simultaneous=2):
    #Call the BNN training with the hyperparams
    DETACHED_PROCESS = 0x00000008
    completed = False
    while not completed:
        instances = 0
        for proc in ps.process_iter():
            try:
                if 'BNN_train.py' in proc.cmdline():
                    if not single_run_mode or proc.cmdline()[-2] == run_id:
                        instances += 1
            except:
                pass
        if instances == 0:
            completed = True
        else:
            time.sleep(1)
    num_running=0
    for m in range(NNs):
        num_running = len(get_running())
        while num_running >= simultaneous:
            num_running = len(get_running())
            time.sleep(1)
        
        cmd = ['D:\\Users\\212687364\\.virtualenvs\\LEAPGPU\\Scripts\\python.exe',
               'BNN_train.py',
               str(num_hidden),
               str(num_neurons),
               str(indep_div),
               '{:.0e}'.format(lr),
               str(m),
               dataset_name,
               run_id,
               str(gpu)
              ]
        p = Popen(cmd)#,shell=False,stdin=None,stdout=None,stderr=None,close_fds=True,creationflags=DETACHED_PROCESS)
    # Wait for all instances of BNN_train.py to complete
    running=[]
    completed = False
    while not completed:
        instances = 0
        for proc in ps.process_iter():
            try:
                if 'BNN_train.py' in proc.cmdline():
                    if not single_run_mode or proc.cmdline()[-2] == run_id:
                        instances += 1
            except:
                pass
        if instances == 0:
            completed = True
        else:
            time.sleep(1)


In [ ]:
run_trial(num_hidden, num_neurons, lr, run_id)

In [ ]:
num_trials = 1000
trial_summaries = []
for trial_no in range(5, num_trials):
    
    run_id = "{:04d}".format(trial_no)
    print(run_id)
    # Select hyperparams
    num_hidden = random.randint(2, 10)
    num_neurons = random.randint(10, 50)
    indep_div = random.randint(200, 300)
    lr = 1e-4
    
    run_trial(num_hidden, num_neurons,indep_div, lr, run_id, dataset_name="NN0042_meas", gpu=False, simultaneous=8)


In [ ]:
from joblib import Parallel, delayed
trials = []
for trial_no in range(1000):
    
    trial_dict= {'run_no':"{:04d}".format(trial_no)}
    # Select hyperparams
    trial_dict['num_hidden'] = random.randint(2, 10)
    trial_dict['num_neurons'] = random.randint(10, 250)
    trial_dict['indep_div'] = random.randint(200, 800)
    lr_exp=random.randint(3, 6)
    trial_dict['lr'] = 10**-lr_exp
    trials.append(trial_dict)
    #run_trial(trial_dict['num_hidden'], trial_dict['num_neurons'],
    #                                   trial_dict['indep_div'], trial_dict['lr'], trial_dict['run_no'],
    #                                   single_run_mode=True, NNs=1, gpu=False)
Parallel(n_jobs=8)(delayed(run_trial)(trial_dict['num_hidden'], trial_dict['num_neurons'],
                                       trial_dict['indep_div'], trial_dict['lr'], trial_dict['run_no'],
                                       single_run_mode=False, NNs=8, gpu=True,
                                      dataset_name='NN0042_X_DFA') for trial_dict in trials)

In [ ]:
data_id = 'NTNU0000_FFT'
curr_model = None

trial_summaries = []
trial_dfs = {}
for i, folder in enumerate(glob.glob("BNN_tuning/{}*".format(data_id))):
    print(i)
    run_id = "{:04d}".format(i)
    hidden_pos = folder.split('_').index('hidden')+1
    stem = ""
    for val in folder.split('_')[:hidden_pos-1]:
        stem += val+'_'
    run_id = stem.split("\\")[1][:-1]
    num_hidden = int(folder.split('_')[hidden_pos])
    num_neurons = int(folder.split('_')[hidden_pos+2])
    lr = float(folder.split('_')[hidden_pos+4])
    #curr_model = load_BNN_dict(data_id, LF_df, num_hidden, num_neurons, lr, run_id, model_dict=curr_model)
    #test_loglosses, short_ll, long_ll = get_BNN_loglosses(*[None]*6, curr_model, return_short_long=True, test_souce='X_val')
    
    histories = []
    run_files = glob.glob("{}/*_hist.csv".format(folder))
    temp_summary = {'id':run_id, 'num_hidden':num_hidden, 'num_neurons':num_neurons, 'learning_rate':lr, 'n_ensemble':len(run_files)}
    val_ll = []
    train_ll = []
    for m, file in enumerate(run_files):
        run_hist = pd.read_csv(file)
        temp_summary['run_length_{}'.format(m)] = len(run_hist)
        temp_summary['min_val_logloss_{}'.format(m)] = np.min(run_hist['val_binary_crossentropy'].values)
        val_ll.append(np.min(run_hist['val_binary_crossentropy'].values))
        temp_summary['min_train_logloss_{}'.format(m)] = np.min(run_hist['binary_crossentropy'].values)
        train_ll.append(np.min(run_hist['binary_crossentropy'].values))
        temp_summary['hist_file_{}'.format(m)] = file
        trial_dfs["{}_{}".format(run_id, m)] = run_hist
    temp_summary['avg_val_binary_crossentropy'] = np.mean(val_ll)
    #temp_summary['en_short_val_binary_crossentropy'] = np.mean(short_ll)
    #temp_summary['en_long_val_binary_crossentropy'] = np.mean(long_ll)
    temp_summary['avg_train_binary_crossentropy'] = np.mean(train_ll)
    #print("Logloss: {:.3f} \t Short Logloss: {:.3f} \t Long Logloss: {:.3f} \t".format(np.mean(val_ll), short_ll, long_ll))
    trial_summaries.append(temp_summary)
summary_df = pd.DataFrame(trial_summaries)

summary_df[['id', 'num_hidden', 'num_neurons', 'learning_rate', 'n_ensemble', 'avg_train_binary_crossentropy','avg_val_binary_crossentropy',
            #'en_short_val_binary_crossentropy', 'en_long_val_binary_crossentropy',
            'hist_file_0']].sort_values('avg_val_binary_crossentropy')